### Task 4.9 - Part 1 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\Jay\Documents\Instacart Basket Analysis'

In [3]:
# import data
df_customers = pd.read_csv(os.path.join(path, '02 Data Sets', 'Original Data', 'customers.csv'), index_col = False)

In [4]:
df_customers.shape

(206209, 10)

In [5]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


#### Wrangling procedures

In [6]:
# rename columns
df_customers = df_customers.rename(columns = {'First Name':'first_name', 'Surnam':'last_name', 'Gender':'gender', 'STATE':'state', 'Age':'age', 'n_dependants':'dependants','fam_status':'family_status'})

In [7]:
df_customers.head()

,user_id,first_name,last_name,gender,state,age,date_joined,dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


##### The customer name(s) may not be needed for this analysis as they have an included user_id, however I'm not certain enough to drop these columns at this point.

#### Consistency checks

In [8]:
# check for mixed data types
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

first_name


In [9]:
# check for missing values
df_customers.isnull().sum()

user_id              0
first_name       11259
last_name            0
gender               0
state                0
age                  0
date_joined          0
dependants           0
family_status        0
income               0
dtype: int64

In [10]:
# check for full duplicate records
df_dups = df_customers[df_customers.duplicated()]
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,dependants,family_status,income


##### No duplicate records indicated, and only first_name has mixed and missing values. I'm now confident to drop this column accordingly.

In [11]:
# drop first_name column
df_customers = df_customers.drop('first_name', 1)

C:\Users\Jay\AppData\Local\Temp/ipykernel_18916/2215633146.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_customers = df_customers.drop('first_name', 1)


In [12]:
df_customers.shape

(206209, 9)

In [13]:
df_customers.head()

,user_id,last_name,gender,state,age,date_joined,dependants,family_status,income
0,26711,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


#### Combine prepared data with customer data

In [14]:
# import prepared data
ords_prods = pd.read_pickle(os.path.join(path, '02 Data Sets', 'Prepared Data', 'ords_prods_merge_agg.pkl'))

In [15]:
ords_prods.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,...,price_range_loc,Busiest day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,spending,spending_flag,frequency,frequency_flag
0,2539329,1,prior,1,2,8,NaN,True,196,1,...,Mid-range product,Regularly busy,Regularly busy,Most_orders,10,New customer,6.367797,Low Spender,20.5,Non-frequent customer
1,2539329,1,prior,1,2,8,NaN,True,14084,2,...,Mid-range product,Regularly busy,Regularly busy,Most_orders,10,New customer,6.367797,Low Spender,20.5,Non-frequent customer
2,2539329,1,prior,1,2,8,NaN,True,12427,3,...,Low-range product,Regularly busy,Regularly busy,Most_orders,10,New customer,6.367797,Low Spender,20.5,Non-frequent customer
3,2539329,1,prior,1,2,8,NaN,True,26088,4,...,Low-range product,Regularly busy,Regularly busy,Most_orders,10,New customer,6.367797,Low Spender,20.5,Non-frequent customer
4,2539329,1,prior,1,2,8,NaN,True,26405,5,...,Low-range product,Regularly busy,Regularly busy,Most_orders,10,New customer,6.367797,Low Spender,20.5,Non-frequent customer


In [16]:
ords_prods['user_id'].dtype

dtype('int64')

In [17]:
df_customers.dtypes

user_id           int64
last_name        object
gender           object
state            object
age               int64
date_joined      object
dependants        int64
family_status    object
income            int64
dtype: object

In [34]:
# Merge above dataframes
df_merged = ords_prods.merge(df_customers, on = 'user_id')

In [35]:
df_merged.shape

(32435059, 34)

In [36]:
df_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,...,frequency,frequency_flag,last_name,gender,state,age,date_joined,dependants,family_status,income
0,2539329,1,prior,1,2,8,NaN,True,196,1,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2539329,1,prior,1,2,8,NaN,True,14084,2,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,2539329,1,prior,1,2,8,NaN,True,12427,3,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2539329,1,prior,1,2,8,NaN,True,26088,4,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,2539329,1,prior,1,2,8,NaN,True,26405,5,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [37]:
# Export data to pkl
df_merged.to_pickle(os.path.join(path, '02 Data Sets','Prepared Data', 'ords_prods_cust.pkl'))